# 4. Example: Download era5 from cloud provider

DataAssimBench provides interfaces to ECWMF ERA5 data hosted on Google Cloud, Amazon Web Services, and Microsoft Azure.

In this example, we'll walk through loading data from Amazon Web Services (AWS) into a dabench data object.

### Part I: Imports and Model Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from dabench import data

In [ ]:
# We can find info the options for the AWS data loader by running help()
help(data.AWS)

To start, let's download air temp data over the state of Colorado for 2020 and 2021.

Note: If you don't specify any of these options, the model object will use defaults, which are air temp at 2 metres for 2020 over Cuba. While you can download global data via this interface, it is not recommended (except for over very short time periods) due to the data volume.

In [ ]:
# The model object is defined in the same way as the model data generators (e.g. Lorenz63).
# But the options vary a bit. In this case we can specify which variables and years to load
# and set a bounding box
aws_loader = data.AWS(variables = ['air_temperature_at_2_metres'], 
                       years = [2020, 2021],
                       min_lat = 36.992426, max_lat = 41.003444, 
                       min_lon = -109.060253, max_lon = -102.041524
                      )

In [ ]:
# To download the data, we can use aws_loader.generate() 
# OR aws_loader.load()
aws_loader.load()
# Also works, but .generate() shows a warning because it 
# is a simple alias for .load()
# aws_loader.generate()

### Part II: Examining and Visualizing Data

The process for accessing and visualizing the data is much the same as it is for the other data objects (e.g. models)

In [ ]:
# In this case, we have 17544 timesteps (24 hours for 731 days, 2020 was a leap year)
print(aws_loader.values.shape)
print(aws_loader.values)

In [ ]:
# We can access those timesteps directly:
print(aws_loader.times)

In [ ]:
# Convert each timestep back to 2D
gridded_values = aws_loader.to_original_dim()
print(gridded_values.shape)

In [ ]:
# Now let's compare air temp in January vs June over Colorado
fig, ax = plt.subplots(1, 2)
fig.suptitle('Air Temp at 2 Metres (K), Colorado')
ax[0].imshow(gridded_values[12], vmin=250, vmax=300)
ax[0].set_title(np.datetime_as_string(aws_loader.times[12], unit='h')); ax[0].set_xlabel('Longitude'); ax[0].set_ylabel('Latitude')
ax[0].set_yticks(ticks=[0, 5, 10, 15], labels=[37, 38.25, 39.5, 40.75]); ax[0].set_xticks(ticks=[0, 10, 20], labels=[-109, -106.5, -104])
ax[1].imshow(gridded_values[3660], vmin=250, vmax=300)
ax[1].set_title(np.datetime_as_string(aws_loader.times[3660], unit='h')); ax[1].set_xlabel('Longitude'); ax[1].set_ylabel('Latitude')
ax[1].set_yticks(ticks=[0, 5, 10, 15], labels=[37, 38.25, 39.5, 40.75]); ax[1].set_xticks(ticks=[0, 10, 20], labels=[-109, -106.5, -104])
fig.tight_layout()
fig.subplots_adjust(top=1.4)
plt.show()

### Part III: Customizing data download options

This interface is highly flexible. ERA5 offers many different variables that can be used. Multiple variables can be loaded at the same time.

You can find the full list of variables here: https://github.com/planet-os/notebooks/blob/master/aws/era5-pds.md#variables

Note that you cannot load forecast variables (denoted by "fc" in the table) and analysis variables ("an") into the same data object. You can load one or more forecast variables OR one or more analysis variables, but not both.

Let's load air pressure and sea surface temperature data over Cuba (the default bounding box) for 1983. Note that you should not included the ".nc" extension in your variable list.

In [ ]:
# Cuba is the default bounding box so we don't need to specify that|
cuba_loader = data.AWS(variables = ['air_pressure_at_mean_sea_level', 'sea_surface_temperature'], 
                       years = [1979]
                      )
cuba_loader.load()

Let's look at the shape of our data

In this case, we have:
- timesteps = 8760 (24 hrs x 365 days)
- x dim = 13
- y dim = 43
- variables = 2

In [ ]:
cuba_grid_vals = cuba_loader.to_original_dim()
print(cuba_grid_vals.shape) 

In [ ]:
# Now let's plot sea surface temp and pressure next to each other in July
fig, ax = plt.subplots(1, 2)
fig.suptitle('Sea Surface Pressure and Temp, Cuba in June 1979')
ax[0].imshow(cuba_grid_vals[4404, :, :, 0])
ax[0].set_title('Pressure, ' + np.datetime_as_string(cuba_loader.times[4404], unit='h')); ax[0].set_xlabel('x'); ax[0].set_ylabel('y')
ax[1].imshow(cuba_grid_vals[4404, :, :, 1])
ax[1].set_title('Temperature, ' + np.datetime_as_string(cuba_loader.times[4404], unit='h')); ax[1].set_xlabel('x'); ax[1].set_ylabel('y')
fig.tight_layout()
fig.subplots_adjust(top=1.4)
plt.show()

A couple things to note:
1. Notice that pressures are lower and temps are higher to the Southwest vs. the Northeast
2. Sea surface tempearture is not defined over land